In [ ]:
#### Collect the tweets of the first five kinds of drugs.
import seaborn as sns
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
import multiprocessing
import time
import re
import pandas as pd
import tqdm 
out_dir = '/data1/data_8t/ivermectin/sentiment/' #Saving path
data_type = {'created_at':'string','id':'string','full_text':'string','user':'object',"extracted_entity_sentiment":'object','drug_entity':'object',"sentiment":'object'}
#Create file name list
import glob
import pandas as pd
import tqdm   ## this is a ticker
data_dir = "/data1/data_8t/ivermectin/sentiment/drug+senti/" 
files = sorted(glob.glob(data_dir+"*.csv"))
file = '|'.join(files) #Change the list to string
keys='vitamin d|Vitamin D2|Vitamin D3|vd3|vd2' #The keywords you want to search with VD as the example

def parallelize_dataframe(df, func, **kwargs):
    #CPUs = multiprocessing.cpu_count()
    num_partitions = 8 # number of partitions to split dataframe
    num_cores =  8 # number of cores on your machine

    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    func = partial(func, **kwargs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
    
def extract(df,keys):
    keys=keys
    
    # #keyword string
    # keywords = pd.read_csv('/data_8t/lwx/keywords/keywords.csv')
    # keys = keywords.Drugs.tolist()  # to list
    # keys = '|'.join(keys) # to string

    #df = df[df['full_text'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)] #filter the data with keys
    # flag=df['symp_entity'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)
    # diff_flag = [not f for f in flag]
    # res = df[diff_flag]
    # res=res.reset_index(drop=True)
    df = df[df['drug_entity'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)] #filter the data with keys
    #print(len(df))
    df1 = df.drop_duplicates(subset=['id']) #drop the duplicates
    return df1


df_empty = pd.DataFrame(columns=["created_at","id","full_text","user",'drug_entity','sentiment'])
for file in tqdm.tqdm(files):
    df = pd.read_csv(file,dtype=data_type, lineterminator='\n',usecols=['created_at','id','full_text','user','drug_entity','sentiment'],low_memory=False, keep_default_na=False)#, iterator=True,chunksize=100000)
    df1=extract(df,keys)
    #df1 = parallelize_dataframe(df, extract)
    df_empty=pd.concat([df_empty,df1])
#keys='vitamin d|Vitamin D2|Vitamin D3|vd3|vd2|vit d|Calcifediol'
df_empty.to_csv(out_dir+keys+".csv",index=False)  #Save the data
print(len(df_empty))
# Decriptive Statistics for the sentiment

import pandas as pd
symp=[]
df = pd.read_csv("/data1/data_8t/ivermectin/sentiment/zinc.csv",lineterminator='\n',low_memory=False, keep_default_na=False)
s=df['sentiment'].tolist()
symp.extend(s)
a=[]
for i in symp:
    b=(i[1:-1])
    if b!='':
       a.append((eval(b)))

b=[]
for i in a:
    if type(i)==tuple:
        i=list(i)
        for ii in i:
            #print(type(ii),ii)
            b.append(ii)
    else:
        #print(type(i),i)
        b.append(i)
dict = {}
for key in b:
    dict[key] = dict.get(key, 0) + 1
print(dict)
#### Extract the number of emotions related to the drugs in each tweet.
def Getsenti_num(twt):
    n=twt.count(',')+1
    return n

def Getpos_num(twt):
    n=twt.count('positive')
    return n

def Getneg_num(twt):
    n=twt.count('negative')
    return n

def Getneu_num(twt):
    n=twt.count('neutral')
    return n

def Senti_Entity(entity):#Extract the "drug" entity in the entity extraction part.
    senti_entity = []
    try:
        for i in eval(entity): 
            if i['type'] == 'Drug' and 'sentiment' in i:
                senti_entity.append(i['sentiment'].lower())
        return senti_entity
    except TypeError: # because some users has no text
        return '[]'

def parall_func(df):
    df['senti_num'] = df['sentiment'].apply(Getsenti_num)
    df['pos_num'] = df['sentiment'].apply(Getpos_num)
    df['neg_num'] = df['sentiment'].apply(Getneg_num)
    df['neu_num'] = df['sentiment'].apply(Getneu_num)
    return df


#pattern = re.compile('|'.join(symptom_keywords),flags=re.I)
# This function is used for multi-threading. The input is the data frame you want to process and the corresponding function.
def parallelize_dataframe(df, func, **kwargs):
    #CPUs = multiprocessing.cpu_count()
    num_partitions = 4 # number of partitions to split dataframe
    num_cores =  4 # number of cores on your machine

    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    func = partial(func, **kwargs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

'''
# find the keywords in the text. if a keyword appears 
# several time in a text, only return one of it.
'''
df = pd.read_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv',lineterminator='\n',low_memory=False)
df = parallelize_dataframe(df, parall_func)
df.to_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv')
#Calculate the proportion of "positive" in the weekly tweets regarding zinc.
import pandas as pd
df=pd.read_csv('zinc.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincn.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincs.csv')
# Calculate the proportion of "positive" in the weekly tweets regarding HCQ.
import pandas as pd
df=pd.read_csv('/data1/data_8t/ivermectin/sentiment/Hydroxychloroquine|HCQ|plaquenil.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqhcqp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('hcq.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqhcqn.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('hcq.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqhcqs.csv')
# Calculate the proportion of "positive" in the weekly tweets regarding Ivermectin.
import pandas as pd
df=pd.read_csv('zinc.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincn.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('zinc.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqzincs.csv')
# Calculate the proportion of "positive" in the weekly tweets regarding Remdesivir.
import pandas as pd
df = pd.read_csv('/data1/data_8t/ivermectin/sentiment/Remdesivir|velkury.csv',lineterminator='\n',low_memory=False)
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqremp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('rem.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqremn.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('rem.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqrems.csv')
import pandas as pd
df=pd.read_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv')
data=df[['created_at','pos_num']]
data.to_csv('vitd_senti.csv',index=False)
# Calculate the proportion of "positive" in the weekly tweets regarding Vitamin D.
data=pd.read_csv('vitd_senti.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
data=pd.read_csv('vitd_senti.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdn.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
data=pd.read_csv('vitd_senti.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitds.csv')
# Calculate the proportion of "positive" in the weekly tweets regarding Ivermectin.
import pandas as pd
df=pd.read_csv('/data1/data_8t/ivermectin/sentiment/ivermectin|Stromectol.csv')
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqiverp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqivern.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqivers.csv')
# Calculate the proportion of "positive" in the weekly tweets regarding Ivermectin.
import pandas as pd
df = pd.read_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv',lineterminator='\n',low_memory=False)
df = parallelize_dataframe(df, parall_func)
df.to_csv('/data1/data_8t/ivermectin/sentiment/vitamin d|Vitamin D2|Vitamin D3|vd3|vd2.csv',index=False)
data=df[['created_at','pos_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdp.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','neg_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitdn.csv')
#And then divide it by the total number of entities per week.

import pandas as pd
#df=pd.read_csv('iver.csv')
data=df[['created_at','senti_num']]
data['created_at'] = pd.to_datetime(data['created_at'])
data.index = data['created_at']
data=data.sort_index()
#del data['created_at']
#data.insert(data.shape[1], 'd', 1)
p=data.resample('W', label='right', closed='right').sum()
p.to_csv('freqvitds.csv')